In [1]:
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D
from tensorflow.keras.layers import Add, Dense, BatchNormalization, Dropout
from tensorflow.keras.layers import ReLU, MaxPool2D, GlobalAvgPool2D, LeakyReLU
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential, save_model, load_model
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
import datetime
import cv2
import os
import tensorflow as tf
import tensorflow.keras
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

from pathlib import Path
from tensorflow.keras.utils import to_categorical
from collections import Counter
from plotly.subplots import make_subplots
from tensorflow.keras.layers.experimental import preprocessing as ps
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle
import glob
print('tf version', tf.__version__)
print('keras version', tf.keras.__version__)
print('gpu is ','available' if tf.config.list_physical_devices('GPU') else 'not available')

tf version 2.8.0
keras version 2.8.0
gpu is  available


## Convulation layer with BatchNormalization

In [2]:
def conv_layer(x, filters, kernel_size, strides=1):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size, 
               strides=strides,
               padding='same',
               use_bias=False)(x)
    x = BatchNormalization()(x)
    return x

## Separable Convulation layer with BatchNormalization

In [3]:
def sep_conv_layer(x, filters, kernel_size, strides=1):
    x = SeparableConv2D(filters=filters,
                        kernel_size=kernel_size,
                        strides=strides,
                        padding='same',
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    return x

## Xception model Entry Flow

In [4]:
def entry_flow(x):
    x = conv_layer(x, filters=32, kernel_size=3, strides=2)
    x = ReLU()(x)
    x = conv_layer(x, filters=64, kernel_size=3)
    tensor = ReLU()(x)
    
    x = sep_conv_layer(tensor, filters=128, kernel_size=3)
    x = ReLU()(x)
    x = sep_conv_layer(x, filters=256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    tensor = conv_layer(tensor, filters=256, kernel_size=1, strides=2)
    
    x = Add()([tensor, x])
    x = ReLU()(x)
    x = sep_conv_layer(x, filters=256, kernel_size=3)
    x = ReLU()(x)
    x = sep_conv_layer(x, filters=256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    tensor = conv_layer(tensor, filters=256, kernel_size=1, strides=2)
    
    x = Add()([tensor, x])
    x = ReLU()(x)
    x = sep_conv_layer(x, filters=512, kernel_size=3)
    x = ReLU()(x)
    x = sep_conv_layer(x, filters=1024, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    tensor = conv_layer(tensor, filters=1024, kernel_size=1, strides=2)
    x = Add()([tensor, x])
    
    return x

## Xception model Middle flow

In [5]:
def middle_flow(tensor):
    for _ in range(4):
        x = ReLU()(tensor)
        x = sep_conv_layer(x, filters=1024, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_layer(x, filters=1024, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_layer(x, filters=1024, kernel_size=3)
        x = ReLU()(x)
        
        tensor = Add()([tensor, x])
    
    return tensor

## Xception model Exit flow

In [6]:
def exit_flow(tensor):
    x = LeakyReLU()(tensor)
    x = sep_conv_layer(x, filters=728, kernel_size=3)
    x = LeakyReLU()(x)
    x = sep_conv_layer(x, filters=512, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    tensor = conv_layer(tensor, filters=512, kernel_size=1, strides=2)
    
    x = Add()([tensor, x])
    x = sep_conv_layer(x, filters=512, kernel_size=3)
    x = LeakyReLU()(x)
    x = sep_conv_layer(x, filters=256, kernel_size=3)
    x = LeakyReLU()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(128, activation=LeakyReLU())(x)
    x = Dense(64, activation=tf.keras.layers.LeakyReLU())(x)
    x = Dropout(0.5)(x)
    x = Dense(units=49, activation='softmax')(x)
    
    return x

## Model creation 

In [7]:
input = Input(shape=[128, 128, 3])

x = entry_flow(input)
x = middle_flow(x)
output = exit_flow(x)

base_model = Model(input, output)

In [8]:
num_of_class = 49
epoch = 75

model = Sequential()
model.add(base_model)

model.compile(loss ='categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

## BDSL Dataset Training 94%

In [9]:
file = open('X_train_128.pkl', 'rb')
X_train = pickle.load(file)

file = open('y_train_128.pkl', 'rb')
y_train = pickle.load(file)

file = open('X_test_128.pkl', 'rb')
X_test = pickle.load(file)

file = open('y_test_128.pkl', 'rb')
y_test = pickle.load(file)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11774, 128, 128, 3)
(11774, 49)
(2940, 128, 128, 3)
(2940, 49)


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 49)                16078025  
                                                                 
Total params: 16,078,025
Trainable params: 16,040,281
Non-trainable params: 37,744
_________________________________________________________________


In [ ]:
H = model.fit(
    x=X_train,
    y=y_train,
    batch_size=58,
    epochs=epoch,
    validation_data=(X_test, y_test))

Epoch 1/75
203/203 [==============================] - 74s 329ms/step - loss: 3.2722 - accuracy: 0.1431 - val_loss: 3.9035 - val_accuracy: 0.0204
Epoch 2/75
203/203 [==============================] - 66s 326ms/step - loss: 1.4847 - accuracy: 0.5476 - val_loss: 3.5019 - val_accuracy: 0.0959
Epoch 3/75
203/203 [==============================] - 66s 326ms/step - loss: 0.7187 - accuracy: 0.7692 - val_loss: 0.8735 - val_accuracy: 0.7122
Epoch 4/75
203/203 [==============================] - 67s 331ms/step - loss: 0.4475 - accuracy: 0.8543 - val_loss: 0.9158 - val_accuracy: 0.7313
Epoch 5/75
203/203 [==============================] - 67s 331ms/step - loss: 0.3027 - accuracy: 0.8999 - val_loss: 0.7248 - val_accuracy: 0.7867
Epoch 6/75
203/203 [==============================] - 68s 333ms/step - loss: 0.2225 - accuracy: 0.9243 - val_loss: 0.8183 - val_accuracy: 0.7816
Epoch 7/75
203/203 [==============================] - 68s 333ms/step - loss: 0.1903 - accuracy: 0.9387 - val_loss: 1.6583 - val_ac

In [ ]:
# Plot training & validation accuracy values
fig, ax = plt.subplots(1,1)
plt.plot(H.history["accuracy"])
plt.plot(H.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
fig, ax = plt.subplots(1,1)
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_pred = np.argmax(model.predict(X_test,batch_size=1), axis=1) 

print('Classification Report')
y_classes = [np.argmax(y, axis=None, out=None) for y in y_test]
report = classification_report(y_classes, y_pred)
print(report)

plt.figure(figsize=(30,30))
cm = confusion_matrix(y_classes, y_pred)
_=sns.heatmap(cm.T, annot=True, fmt='d', cbar=True, square=True)
plt.xlabel('Truth')
plt.ylabel('Predicted')